<a href="https://colab.research.google.com/github/khyati51/Hate-Speech-Recognition/blob/main/Hate_Speech_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
pip install tensorflow

In [4]:
!pip install --upgrade gensim

In [5]:
column_names = ['Comment Text', 'Comment Type']
train = pd.read_csv('/content/NLP_ass_train.tsv',sep='\t',names=column_names)
valid = pd.read_csv('/content/NLP_ass_valid.tsv',sep='\t',names=column_names)
test = pd.read_csv('/content/NLP_ass_test.tsv',sep='\t',names=column_names)
train

,Comment Text,Comment Type
0,we cannot continue calling ourselves feminists...,normal
1,nawt yall niggers ignoring me,normal
2,<user> i am bit confused coz chinese ppl can n...,hatespeech
3,and this is why i end up with nigger trainee d...,hatespeech
4,nogs jews and dykes how enriching,offensive
...,...,...
15378,this vile racist ghetto ass dumber than a reta...,offensive
15379,when i first got on here and said i hate trump...,offensive
15380,was macht der moslem wenn der zion gegen seine...,normal
15381,it is awful look at world demographics asians ...,hatespeech


##Text Pre Processing

In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [7]:
# Download required NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
def preprocess_text(text):
    #print("Original Text:", text)

    # Remove HTML tags
    text = re.sub('<[^>]*>', '', text)

    print("Text after removing HTML tags:", text)

    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub('[^a-zA-Z]', ' ', text).lower()

    print("Text after removing non-alphabetic characters and converting to lowercase:", text)

    # Tokenize the text
    words = word_tokenize(text)

    print("Text after tokenization:", words)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    print("Text after removing stop words:", words)

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    print("Text after Lemmatization:", words)

    # Combine words back into a single string
    preprocessed_text = ' '.join(words)

    print("Final pre-processed text:", preprocessed_text)

    return preprocessed_text

In [9]:
preprocess_text("This is Khyati Gupta, I am making this project:, related to NLP. I WILL BE learning all the techniques used in NLP and about RNN.")

Text after removing HTML tags: This is Khyati Gupta, I am making this project:, related to NLP. I WILL BE learning all the techniques used in NLP and about RNN.
Text after removing non-alphabetic characters and converting to lowercase: this is khyati gupta  i am making this project   related to nlp  i will be learning all the techniques used in nlp and about rnn 
Text after tokenization: ['this', 'is', 'khyati', 'gupta', 'i', 'am', 'making', 'this', 'project', 'related', 'to', 'nlp', 'i', 'will', 'be', 'learning', 'all', 'the', 'techniques', 'used', 'in', 'nlp', 'and', 'about', 'rnn']
Text after removing stop words: ['khyati', 'gupta', 'making', 'project', 'related', 'nlp', 'learning', 'techniques', 'used', 'nlp', 'rnn']
Text after Lemmatization: ['khyati', 'gupta', 'making', 'project', 'related', 'nlp', 'learning', 'technique', 'used', 'nlp', 'rnn']
Final pre-processed text: khyati gupta making project related nlp learning technique used nlp rnn


'khyati gupta making project related nlp learning technique used nlp rnn'

In [10]:
%%time
train['preprocessed_text'] = train['Comment Text'].apply(preprocess_text)
test['preprocessed_text'] = test['Comment Text'].apply(preprocess_text)
valid['preprocessed_text'] = valid['Comment Text'].apply(preprocess_text)

Streaming output truncated to the last 5000 lines.
Text after removing HTML tags: i am such a shady hoe omg
Text after removing non-alphabetic characters and converting to lowercase: i am such a shady hoe omg
Text after tokenization: ['i', 'am', 'such', 'a', 'shady', 'hoe', 'omg']
Text after removing stop words: ['shady', 'hoe', 'omg']
Text after Lemmatization: ['shady', 'hoe', 'omg']
Final pre-processed text: shady hoe omg
Text after removing HTML tags:  damn he actually retarded
Text after removing non-alphabetic characters and converting to lowercase:  damn he actually retarded
Text after tokenization: ['damn', 'he', 'actually', 'retarded']
Text after removing stop words: ['damn', 'actually', 'retarded']
Text after Lemmatization: ['damn', 'actually', 'retarded']
Final pre-processed text: damn actually retarded
Text after removing HTML tags: this bitch kenny got my key i can ’ t emuss lock the damn door
Text after removing non-alphabetic characters and converting to lowercase: this b

In [11]:
train

,Comment Text,Comment Type,preprocessed_text
0,we cannot continue calling ourselves feminists...,normal,continue calling feminist right womxn arent ad...
1,nawt yall niggers ignoring me,normal,nawt yall nigger ignoring
2,<user> i am bit confused coz chinese ppl can n...,hatespeech,bit confused coz chinese ppl access twitter th...
3,and this is why i end up with nigger trainee d...,hatespeech,end nigger trainee doctor speak properly lack ...
4,nogs jews and dykes how enriching,offensive,nog jew dyke enriching
...,...,...,...
15378,this vile racist ghetto ass dumber than a reta...,offensive,vile racist ghetto as dumber retarded tree sna...
15379,when i first got on here and said i hate trump...,offensive,first got said hate trump considered troll peo...
15380,was macht der moslem wenn der zion gegen seine...,normal,macht der moslem wenn der zion gegen seinen pr...
15381,it is awful look at world demographics asians ...,hatespeech,awful look world demographic asian fucking eve...


In [12]:
# Encode Comment Type labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train['label_encoded'] = label_encoder.fit_transform(train['Comment Type'])
valid['label_encoded'] = label_encoder.transform(valid['Comment Type'])
test['label_encoded'] = label_encoder.transform(test['Comment Type'])

##Word2Vec Embedding

In [13]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
# word2vec_path = ('/content/drive/MyDrive/GoogleNews-vectors-negative300 (1).bin.gz')
import os

word2vec_path = '/content/drive/MyDrive/GoogleNews-vectors-negative300 (1).bin.gz'

if os.path.isfile(word2vec_path):
    print("File exists and is accessible.")
else:
    print("File does not exist or is not accessible.")

# def get_google_word2vec_embeddings(data, word2vec_path):
#     # Load the Google News Word2Vec model
#     word2vec_model = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
#     tokenized_sentences = [sentence.split() for sentence in data]
#     embeddings = []

#     for sentence in tokenized_sentences:
#         sentence_embeddings = []
#         for word in sentence:
#             if word in word2vec_model:
#                 sentence_embeddings.append(word2vec_model[word])
#         if sentence_embeddings:
#             embeddings.append(np.mean(sentence_embeddings, axis=0))
#         else:
#             embeddings.append(np.zeros(300))  # If no embeddings found, use zeros

#     return np.array(embeddings)
def get_embeddings(data, batch_size=100):
  word2vec_path = '/content/drive/MyDrive/GoogleNews-vectors-negative300 (1).bin.gz'
  try:
    word2vec_model = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
    print("Word2Vec model loaded successfully.")
  except Exception as e:
    print(f"Error loading Word2Vec model: {e}")
    return None

  tokenized_sentences = [sentence.split() for sentence in data]
  embeddings = []

  for sentence in tokenized_sentences:
      sentence_embeddings = []
      for word in sentence:
          if word in word2vec_model:
              sentence_embeddings.append(word2vec_model[word])
      if sentence_embeddings:
          embeddings.append(np.mean(sentence_embeddings, axis=0))
      else:
          embeddings.append(np.zeros(300))

  return np.array(embeddings)

File exists and is accessible.


In [14]:
    # Generate input features using Word2Vec embeddings
#X_train = get_embeddings(train['preprocessed_text'])
#X_valid = get_embeddings(valid['preprocessed_text'])
#X_test = get_embeddings(test['preprocessed_text'])
#y_train = train['label_encoded'].values
#y_valid = valid['label_encoded'].values
#y_test = test['label_encoded'].values
#X_train

##CNN Model

In [27]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score, f1_score
from gensim.models import KeyedVectors

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam

def create_cnn_model(input_dim, num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(input_dim, 1)),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])


    return model

def train_nn_with_word2vec(train, valid, test, word2vec_path, num_classes, epochs=20, batch_size=256):

    # Generate input features using Word2Vec embeddings
    X_train = get_embeddings(train['preprocessed_text'])
    X_valid = get_embeddings(valid['preprocessed_text'])
    X_test = get_embeddings(test['preprocessed_text'])

    y_train = train['label_encoded'].values
    y_valid = valid['label_encoded'].values
    y_test = test['label_encoded'].values

    from sklearn.preprocessing import StandardScaler

# Initialize the scaler
    scaler = StandardScaler()

# Fit and transform the training data
    X_train_scaled = scaler.fit_transform(X_train)

# Transform the validation and test data
    X_valid_scaled = scaler.transform(X_valid)
    X_test_scaled = scaler.transform(X_test)

    model = create_cnn_model(X_train_scaled.shape[1], num_classes)
    print(model.summary() )
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Define callbacks to save the best model
    checkpoint_callback = ModelCheckpoint('best_model_nn.h5', monitor='val_loss', save_best_only=True, verbose=1)

    # Train the model
    history = model.fit(X_train_scaled, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(X_valid_scaled, y_valid),
                        callbacks=[checkpoint_callback])

    # Load the best model
    model.load_weights('best_model_nn.h5')

    # Evaluate on test data
    test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)
    print(f'Test Loss: {test_loss:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')

     # Predict on test data
    y_pred = model.predict(X_test_scaled)
    y_pred = np.argmax(y_pred, axis=1)

    # Calculate macro-F1 score
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    print(f'Macro-F1 Score: {macro_f1:.4f}')

    return model, test_accuracy, macro_f1

In [29]:
trained_model, test_accuracy, macro_f1 = train_nn_with_word2vec(train, valid, test, word2vec_path, num_classes=3)

Word2Vec model loaded successfully.
Word2Vec model loaded successfully.
Word2Vec model loaded successfully.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 298, 64)           256       
                                                                 
 max_pooling1d (MaxPooling1  (None, 149, 64)           0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 149, 64)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 147, 128)          24704     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 73, 128)           0         
 g1D)                                                            
              

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


61/61 [==============================] - ETA: 0s - loss: 0.9534 - accuracy: 0.5542
Epoch 2: val_loss improved from 0.98823 to 0.93832, saving model to best_model_nn.h5
61/61 [==============================] - 30s 485ms/step - loss: 0.9534 - accuracy: 0.5542 - val_loss: 0.9383 - val_accuracy: 0.5624
Epoch 3/20
61/61 [==============================] - ETA: 0s - loss: 0.9175 - accuracy: 0.5791
Epoch 3: val_loss improved from 0.93832 to 0.92579, saving model to best_model_nn.h5
61/61 [==============================] - 30s 481ms/step - loss: 0.9175 - accuracy: 0.5791 - val_loss: 0.9258 - val_accuracy: 0.5734
Epoch 4/20
61/61 [==============================] - ETA: 0s - loss: 0.8933 - accuracy: 0.5953
Epoch 4: val_loss improved from 0.92579 to 0.89968, saving model to best_model_nn.h5
61/61 [==============================] - 30s 480ms/step - loss: 0.8933 - accuracy: 0.5953 - val_loss: 0.8997 - val_accuracy: 0.5864
Epoch 5/20
61/61 [==============================] - ETA: 0s - loss: 0.8641 - a

In [30]:
# Task 2: Calculate Intersection of Sentences

def calculate_intersection(train, valid, test):
    train_sentences = set(train['preprocessed_text'])
    valid_sentences = set(valid['preprocessed_text'])
    test_sentences = set(test['preprocessed_text'])

    intersection_train = train_sentences.intersection(test_sentences)
    intersection_valid = valid_sentences.intersection(test_sentences)

    print(f'Number of common sentences between train and test sets: {len(intersection_train)}')
    print(f'Number of common sentences between validation and test sets: {len(intersection_valid)}')

# Calculate and print intersection
calculate_intersection(train, valid, test)

Number of common sentences between train and test sets: 20
Number of common sentences between validation and test sets: 4


##RNN Model

In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Preparing the data
max_words = 10000
max_len = 100

# Tokenize text and convert to sequences
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train['preprocessed_text'])

X_train = tokenizer.texts_to_sequences(train['preprocessed_text'])
X_valid = tokenizer.texts_to_sequences(valid['preprocessed_text'])
X_test = tokenizer.texts_to_sequences(test['preprocessed_text'])

# Pad sequences to max_seq_length
X_train = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_valid = pad_sequences(X_valid, maxlen=max_len, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the validation and test data
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Convert labels to one-hot encoding
y_train = to_categorical(train['label_encoded'])
y_valid = to_categorical(valid['label_encoded'])
y_test = to_categorical(test['label_encoded'])

In [35]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

# Label encode the target variable
le = LabelEncoder()
train['label_encoded'] = le.fit_transform(train['Comment Type'])
valid['label_encoded'] = le.transform(valid['Comment Type'])
test['label_encoded'] = le.transform(test['Comment Type'])

# Convert labels to numpy arrays
y_train = np.array(train['label_encoded'])
y_valid = np.array(valid['label_encoded'])
y_test = np.array(test['label_encoded'])

# Tokenize the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['Comment Text'])

X_train = tokenizer.texts_to_sequences(train['Comment Text'])
X_test = tokenizer.texts_to_sequences(test['Comment Text'])
X_valid = tokenizer.texts_to_sequences(valid['Comment Text'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)
# Initialize the scaler
scaler = StandardScaler()

#t and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

#ansform the validation and test data
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Create the model
model = Sequential()
model.add(Embedding(max_words, 256, input_length=max_len))
model.add(SimpleRNN(128))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, to_categorical(y_train), validation_data=(X_valid_scaled, to_categorical(y_valid)),
                    epochs=10, batch_size=256, callbacks=[early_stop])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, to_categorical(y_test))

print("Test accuracy:", accuracy)

Epoch 1/10
61/61 [==============================] - 35s 453ms/step - loss: 1.1013 - accuracy: 0.3806 - val_loss: 1.0874 - val_accuracy: 0.4053
Epoch 2/10
61/61 [==============================] - 17s 281ms/step - loss: 1.0908 - accuracy: 0.3990 - val_loss: 1.0884 - val_accuracy: 0.4053
Epoch 3/10
61/61 [==============================] - 18s 300ms/step - loss: 1.0870 - accuracy: 0.4054 - val_loss: 1.0893 - val_accuracy: 0.4006
Epoch 4/10
61/61 [==============================] - 18s 290ms/step - loss: 1.0870 - accuracy: 0.4013 - val_loss: 1.0946 - val_accuracy: 0.4058
Epoch 5/10
61/61 [==============================] - 18s 290ms/step - loss: 1.0858 - accuracy: 0.4045 - val_loss: 1.0887 - val_accuracy: 0.4027
Epoch 6/10
61/61 [==============================] - 17s 278ms/step - loss: 1.0847 - accuracy: 0.4069 - val_loss: 1.0883 - val_accuracy: 0.4037
Epoch 7/10
61/61 [==============================] - 17s 279ms/step - loss: 1.0851 - accuracy: 0.4067 - val_loss: 1.0874 - val_accuracy: 0.4037

In [33]:
X_train

array([[   0,    0,    0, ..., 6706,  865, 4818],
       [   0,    0,    0, ...,   86, 2784,   49],
       [   0,    0,    0, ...,  665,  665,  665],
       ...,
       [   0,    0,    0, ...,  441, 8589, 9990],
       [   0,    0,    0, ...,  530,    9,  188],
       [   0,    0,    0, ...,  881, 3481,  747]], dtype=int32)

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define RNN model
def create_rnn_model(vocab_size, max_seq_length, num_classes):
    model = Sequential([
        Embedding(vocab_size, 300, input_length=max_seq_length),  # Using larger embedding size
        LSTM(256, dropout=0.5, recurrent_dropout=0.3, return_sequences=True),  # Bidirectional LSTM with more units
        LSTM(128, dropout=0.3, recurrent_dropout=0.2),  # Additional LSTM layer
        Dense(128, activation='relu'),  # Increased dense layer size
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Create RNN model
rnn_model = create_rnn_model(max_words, max_len, len(label_encoder.classes_))
rnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Define callbacks for early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_rnn.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Train the model
history = rnn_model.fit(X_train, y_train,
                        epochs=10,
                        batch_size=256,
                        validation_data=(X_valid, y_valid),
                        callbacks=[early_stopping, model_checkpoint])

# Evaluate on test set
test_loss, test_accuracy = rnn_model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/10
 2/61 [..............................] - ETA: 8:50 - loss: 1.0980 - accuracy: 0.3359 

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(X_train)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Preparing the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['preprocessed_text'])

X_train = tokenizer.texts_to_sequences(train['preprocessed_text'])
X_test = tokenizer.texts_to_sequences(test['preprocessed_text'])
X_valid = tokenizer.texts_to_sequences(valid['preprocessed_text'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

y_train = to_categorical(train['label_encoded'])
y_test = to_categorical(test['label_encoded'])
y_valid = to_categorical(valid['label_encoded'])

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional, LSTM, BatchNormalization

# Build the model
#model = Sequential()
#model.add(Embedding(max_words, 256, input_length=max_len))
#model.add(Bidirectional(LSTM(128, return_sequences=True)))
#model.add(Dropout(0.5))
#model.add(Bidirectional(LSTM(128)))
#model.add(Dropout(0.5))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(3, activation='softmax'))
#
## Compile the model
#model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


# Build the model
#model = Sequential()
#model.add(Embedding(max_words, 256, input_length=max_len))
#model.add(SimpleRNN(128))
#model.add(Dense(3, activation='softmax'))

# Compile the model
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Build the model
model = Sequential()
model.add(Embedding(max_words, 256, input_length=max_len))
model.add(Bidirectional(SimpleRNN(128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Bidirectional(SimpleRNN(128)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=256)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)